# Optimisation of the reservoir rule curve
In this Notebook we will see why and how to optimise the **rule curve** of a water reservoir system. A rule curve is the representation of the different reservoir operating policies **across the year**. The year is divided into discrete intra-year time periods, for instance days, each period having a specified reservoir operating policy, i.e a function that returns the release volume based on the conditions of the reservoir system (for instance, the reservoir storage, the demand forecast, the time of year, etc.) at that time. In this way, each reservoir operating policy adapts to different conditions across the year, e.g. to seasonal changes of reservoir inflows and demans. Equally as the operating policy, the rule curve is initally optimized to maximize the long-term benefits using (sufficiently long) historical time series or model projections.

<left><img src="../../util/images/Dam4.gif" width = "500px"><left>
    
Once again we consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir rulve curve that optimizes the **long-term** (several years) system performance. We use the historical time series of inflows and water demand to estimate such long-term performance. The underpinning assumption here is that the system forcings observed over the past years are representative of the forcings that will drive the system in the future (if this assumption is not sensible, for instance because of ongoing changes that will likely impact the hydrological regime or demand pattern, then one may use model projections of inflow and demand in place of historical observations) 
<left> <img src="../../util/images/system_representation_IO1.png" width = "600px"><left>

We will use a simple form of rule curve where the reservoir release is a only determined by the storage value, as in the Figure below. At normal operation, i.e. at normal storage values, the average historical release is released. Higher storage values are associated to higher releases, which is useful for flood control purposes, whereas at critically low storage values less water is released to reduce the risk of future water shortages [(Loucks et al., 1981)](https://link.springer.com/book/10.1007/978-3-319-44234-1).
<left><img src="../../util/images/Rule_curve.png" width = "800px"><left>

## Import libraries
To run this notebook we need to import some libraries:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer # Import the optimizer

## Defining the shape of the rule curve
As anticipated, in this notebook we will use a rule curve that determines the reservoir release accross the year based on the storage value only. In particular, we will use a piece-wise linear function of the storage as function of the day of the year. The function is implemented in a series of submodules, that we can import with the following code:

In [2]:
from Modules.Interactive_rule_curves import Interactive_rule_curve,Interactive_rule_curve_manual,Interactive_rule_curve_auto
from irons.Functions.Data_management.Read_data import read_csv_data
from irons.Functions.Data_management.day2week2month import day2week
from irons.Functions.Reservoir_operating_policy.Operating_policy import policy_function
from irons.Functions.Reservoir_operating_policy.Operating_rule_curves import rule_curve

In our code, the rule curve rescaled storage values by the reservoir active capacity, so it varies between 0 (dead storage) and 1 (full storage).
The piece-wise linear function representing the rule curve is delineated by 3 points corresponding to the following dates: 1 Apr, 1 Aug and 1 Dec. The rule curve returns a constant release (the average historical release) when the storages stay in the Normal Storage band (between s1 and s2); the release is reduced if the storage is in the Critical Storage band (below s1), or increased if it is in the Flood Storage band (above s2). Let's now attribute a (tentative) value of s1 and s2 in the 3 points, so that we can visualise the rule curve. For example:

In [3]:
# System characteristics
d             = 15 # ML/week - water demand (we assume as constant) 
env_min       = 4 # ML/week - environmental compensation flow
Qreg_rel_mean = d # ML/week - the (long-term) mean release = demand  

# System constraints
Qreg_rel_min = env_min # ML/week - the release at minimum storage 
Qreg_rel_max = 40 # ML/week - the maximum release capacity 

s_min   = 0 # ML - minimum storage (set to zero for now)
s_max   = 150 #  ML - maximum storage (=reservoir capacity) 

### Rule curve parameters ###
# Points defining the curves 
# (linear interpolation will be applied to define the curves between the points)
curve_dates = ['1 Apr',    '1 Aug',    '1 Dec',    '31 Mar'] # for 1 Apr and 31 Mar we apply the same value
s_0         = [s_min/s_max,s_min/s_max,s_min/s_max,s_min/s_max]
s_1         = [0.6,        0.3,        0.2,        0.6] 
s_2         = [0.9,        0.6,        0.5,        0.9] 
s_3         = [s_max/s_max,s_max/s_max,s_max/s_max,s_max/s_max]
# Points defining the release policy across the year 
# (linear interpolation will be applied to define the curves between the points)
rule_dates  = ['21 Mar',     '21 Jun',     '21 Sep',     '21 Dec']
u_0         = [Qreg_rel_min, Qreg_rel_min, Qreg_rel_min, Qreg_rel_min]
u_1         = [Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean]
u_2         = [Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean]
u_3         = [Qreg_rel_max, Qreg_rel_max, Qreg_rel_max, Qreg_rel_max]

param = {'curves':{'year_date'    : curve_dates,
                   'storage_frac' : [s_0,s_1,s_2,s_3]},
         'rules' :{'year_date' : rule_dates,
                   'release'   : [u_0,u_1,u_2,u_3]}} # max release
s_yday,r_yday = rule_curve(param)

We can now plot an interactive figure of the rule curve together with the plots of the operating policity functions in 1 Apr, 1 Aug and 1 Dec with the following code. Use the sliders to modify the parameters of the rule curve at those 3 dates and see how all the rule curve and the operating policy functions change.

In [4]:
fig_1a,fig_2a,fig_2b,fig_2c,s_1_0_slider,s_1_1_slider,s_1_2_slider = Interactive_rule_curve(
                                                                           policy_function,rule_curve, 
                                                                           s_min, s_max, 
                                                                           Qreg_rel_mean, Qreg_rel_min, Qreg_rel_max,
                                                                           s_1,s_2)

Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.HBox([s_1_0_slider,s_1_1_slider,s_1_2_slider],
                  layout=Box_layout), fig_1a],layout=Box_layout),
              widgets.HBox([fig_2a,fig_2b,fig_2c],layout=Box_layout)],layout=Box_layout)

## Optimising the rule curve by trial and error (manual optimisation)
In this section we will refine the parameters of the rule curve (value of s1 and s2 at 1 Apr, 1 Aug and 1 Dec) by trying to increase the system performance when simulated against the historical inflows, evaporation and water demand data. 
### Loading historical inflows, evaporation and water demand data
Let's assume we want to look at 100 weeks from 2014 to 2015, and load the evaporation, inflow and demand observations for this period from a file.

In [5]:
N = 100 # weeks
### Load evaporation data ###
inputs_folder_path = 'Inputs/'
clim_data_file = 'clim_data_2014_15.csv'
date_day, clim_data_day  = read_csv_data(inputs_folder_path, clim_data_file)
date, e_data_week, e_data_week_cum = day2week(N,date_day,clim_data_day[:,0])
### Load inflow data ###
I_data_file = 'inflow_data_2014_15.csv'
date_day, I_data_day = read_csv_data(inputs_folder_path, I_data_file)
date, I_data_week, I_data_week_cum = day2week(N,date_day,I_data_day[:,0])
### Load demand data ###
d_data_file = 'demand_data_2014_15.csv'
date_day, d_data_day = read_csv_data(inputs_folder_path, d_data_file)
date, d_data_week, d_data_week_cum = day2week(N,date_day,d_data_day)

Plot the inflow time series:

In [6]:
# Axis characteristics
x_sc_1 = DateScale();y_sc_1 = LinearScale(min=0,max=40)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
evap_plot = plt.bar(date,e_data_week[:,0],colors=['green'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['evaporation'], display_legend = True)
inflow_plot = plt.bar(date,I_data_week[:,0],colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
# Figure characteristics
fig_1 = plt.Figure(marks = [inflow_plot,evap_plot],title = 'Inflow', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '900px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1])

Plot the demand time series:

In [7]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(date,d_data_week[:,0],colors=['gray'],stroke = 'lightgray',opacities = [0.7]*(N+1), 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '900px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1b])

### Definition of reservoir simulation function and initial storage
Next, we need to import the iRONs function that implements the reservoir simulation (this function iteratively applies the mass balance equation so to reconstruct the temporal evolution of the reservoir variables over the simulation period). We also need to specify the initial storage volume to start the simulation.

In [8]:
### Import the reservoir simulation function ###
from irons.Functions.Reservoir_system_simulation.Res_sys_sim import Res_sys_sim
### Define initial storage for simulation ###
s_ini = 100 # ML - initial storage volume

### Definition of the system objectives
Last, we need to define the objectives that we want to be maximised by the rule curve. As anticipated in the Introduction of this Notebook, our illustrative reservoir is operated to support domestic supply while maintaining the reservoir level above a prescribed target (this could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment).
We will pursue the first objective by minimising the following Total Squared Deficit (TDC) with respect to the historical water demand:

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-u(t) \ ) \ ]^2 $$

where N is the length of the simulation period and d(t) is the water demand for each time-interval in that period. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

We are also interested in minimising the chances that the reservoir level goes below a minimum threshold. We measure how well this criterion is satisfied by the following Critical Storage Violation (CSV) function:

$$CSV = \sum_{t=1}^{N} [ \ max ( \ cs - s(t) , \ 0) \ ] $$

where, again, N is the length of the simulation period, s is the reservoir storage, and cs is the minimum reservoir storage threshold that should preferably not be transpassed. For our case, let's set this threshold to 40 ML.

In [9]:
cs = np.array([40]*(N+1)) # (ML)  minimum reservoir storage threshold

### Determining the optimal rule curve via interactive visualisation

Now use the sliders to modify the parameters of the rule curve in a way that minimises the Total Squared Deficit (TSD) and the Minimum Storage Violation (MSV).

In [10]:
d = d_data_week[1:,0]# d + np.zeros(N)
fig_1a,fig_1b,fig_1c,s_1_0_slider,s_1_1_slider,s_1_2_slider = Interactive_rule_curve_manual(
                                                                           Res_sys_sim,policy_function,rule_curve,
                                                                           date,
                                                                           I_data_week[1:,0],e_data_week[1:,0], 
                                                                           s_ini, s_min, s_max, 
                                                                           Qreg_rel_mean, Qreg_rel_min, Qreg_rel_max,
                                                                           cs, d)

Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.HBox([s_1_0_slider,s_1_1_slider,s_1_2_slider],
                  layout=Box_layout), fig_1a],layout=Box_layout),fig_1b,fig_1c],layout=Box_layout)

## From manual to automatic optimization 
As we have seen, when we deal with two conflicting objectives, we cannot find a solution that optimise both simoultaneously. If we prioritize one objective, the other one is deteriorated: there is a trade-off between the two. It would then be interesting to explore this tradeoff, and find a set of operating policies that produce different optimal combinations of the two objectives. However, this is too cumbersome to do manually, so we can use a multi-objective optimisation algorithm to do that for us. The algorithm will automatically test a very large number of combinations of the rule parameters (value of s1 and s2 at 1 Apr, 1 Aug and 1 Dec) until it finds a set of combinations that realise (approximately) optimal tradeoffs.

Here we use a multi-objective optimisation algorithm called NSGAII, which is implemented in the Python Platypus package. For more information about these methods and tools, see [Deb et al, 2002](https://ieeexplore.ieee.org/document/996017) and the [Platypus webpage](https://platypus.readthedocs.io). The code to run the optimisation is the following:

In [11]:
s_2_inc = 0.3
def auto_optim(vars):
    
    s_1 = [vars[0],vars[1],vars[2],vars[0]]
    s_2 = [s_1[0]+s_2_inc,s_1[1]+s_2_inc,s_1[2]+s_2_inc,s_1[3]+s_2_inc]
    
    param = {'curves':{'year_date'    : curve_dates,
                       'storage_frac' : [s_0,s_1,s_2,s_3]},

             'rules' :{'year_date' : rule_dates,
                       'release'   : [u_0,u_1,u_2,u_3]}} # max release

    Qreg = {'releases' : {'type'  : 'rule curve',
                          'input' : [policy_function, rule_curve],
                          'param' : param},
            'inflows' : [],
            'rel_inf' : []}
    
    Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(date[1:],
                                               I_data_week[1:,0], e_data_week[1:,0], 
                                               s_ini, s_min, s_max, 
                                               env_min, d, 
                                               Qreg)
    
    TSD = (np.sum((np.maximum(d_data_week-u,[0]*N))**2)).astype('int')
    CSV = (np.sum((np.maximum(cs-s,[0]*(N+1))))).astype('int')
    
    return [TSD, CSV]

problem = Problem(3,2)
Real0   = Real(0, 1-s_2_inc); Real1 = Real(0, 1-s_2_inc); Real2 = Real(0, 1-s_2_inc)

problem.types[:]       = [Real0] + [Real1] + [Real2]
problem.function       = auto_optim

population_size = 50
algorithm       = NSGAII(problem,population_size)
algorithm.run(500) # Number of iterations

results1_optim = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

sol_optim = [algorithm.result[i].variables for i in range(population_size)]

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot, called Pareto front, which displays the combination of the two objective values in correspondence to a set of optimised solutions. Click on one point in the Pareto front to visualise the operating policy that generates that performance, and associated storage time series.  What do you think would be a balanced solution?

In [12]:
fig_pf, fig_2a,fig_2b,fig_2c = Interactive_rule_curve_auto(Res_sys_sim,policy_function,rule_curve,
                                                           date,
                                                           I_data_week[1:,0],e_data_week[1:,0], 
                                                           s_ini, s_min, s_max, 
                                                           Qreg_rel_mean, Qreg_rel_min, Qreg_rel_max,
                                                           cs, d,
                                                           results1_optim,results2_optim,sol_optim)

Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [fig_pf, fig_2a],layout=Box_layout),fig_2b,fig_2c],layout=Box_layout)

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

Loucks D. P. et al (1981) Water resource systems planning and analysis, Prentice-Hall.